# Streaming 流式传输

## Agent 进度

In [2]:
from langchain.agents import create_agent


def get_weather(city: str) -> str:
    """Get weather for a given city."""

    return f"It's always sunny in {city}!"

agent = create_agent(
    model="gpt-5-nano",
    tools=[get_weather],
)
for chunk in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in Shenzhen?"}]},
    stream_mode="updates",
):
    for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")

step: model
content: [{'type': 'tool_call', 'name': 'get_weather', 'args': {'city': 'Shenzhen'}, 'id': 'call_nzbgpKpstJbRKyT4u40MrdiI'}]
step: tools
content: [{'type': 'text', 'text': "It's always sunny in Shenzhen!"}]
step: model
content: [{'type': 'text', 'text': 'Shenzhen weather: It’s always sunny. \n\nWould you like real-time details (temperature, humidity, or the weekly forecast)? I can fetch those.'}]


## LLM tokens

In [6]:
from langchain.agents import create_agent


def get_weather(city: str) -> str:
    """Get weather for a given city."""

    return f"It's always sunny in {city}!"

agent = create_agent(
    model="gpt-5-nano",
    tools=[get_weather],
)
for token, metadata in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in Shenzhen?"}]},
    stream_mode="messages",
):
    # print(f"{token.content}", end="", flush=True)
    print(f"node: {metadata['langgraph_node']}")
    print(f"content: {token.content_blocks}")
    print("\n")

It's always sunny in Shenzhen!Shenzhen: It’s always sunny according to the current forecast. 

Would you like more details (temperature, humidity, wind, precipitation chance) for today or the next 7 days?

## Custom updates 自定义更新

In [10]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer  


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()  
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="openai:gpt-4",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="custom"
):
    print(chunk)

Looking up data for city: San Francisco
Acquired data for city: San Francisco


## Stream multiple modes 流式传输多个模式


In [11]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="gpt-5-nano",
    tools=[get_weather],
)

for stream_mode, chunk in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["updates", "custom"]
):
    print(f"stream_mode: {stream_mode}")
    print(f"content: {chunk}")
    print("\n")

stream_mode: updates
content: {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 217, 'prompt_tokens': 132, 'total_tokens': 349, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CXnrd741Wdkk62gJ0VexRYNQhMfM7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--28539e59-a1c2-450b-95d3-8ceb02c4cea6-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_I1CKwq3fhCNiiiOAcHvgh848', 'type': 'tool_call'}], usage_metadata={'input_tokens': 132, 'output_tokens': 217, 'total_tokens': 349, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}})]}}


